In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv("quotes_dataset.csv")

In [26]:
df.head()

,Number,Quote,Author
0,1,The only thing we have to fear is fear itself.,Franklin D. Roosevelt
1,2,The truth will set you free.,The Bible
2,3,To be yourself in a world that is constantly t...,Ralph Waldo Emerson
3,4,"Success is not final, failure is not fatal: It...",Winston S. Churchill
4,5,The only way to do great work is to love what ...,Steve Jobs


In [27]:
quotes = df['Quote']

In [28]:
quotes.head()

0       The only thing we have to fear is fear itself.
1                         The truth will set you free.
2    To be yourself in a world that is constantly t...
3    Success is not final, failure is not fatal: It...
4    The only way to do great work is to love what ...
Name: Quote, dtype: object

In [29]:
quotes.str.lower()

0         the only thing we have to fear is fear itself.
1                           the truth will set you free.
2      to be yourself in a world that is constantly t...
3      success is not final, failure is not fatal: it...
4      the only way to do great work is to love what ...
                             ...                        
720            believe you can and you're halfway there.
721    the mind is everything. what you think you bec...
722    i have not failed. i've just found 10,000 ways...
723    a journey of a thousand miles begins with a si...
724          it always seems impossible until it's done.
Name: Quote, Length: 725, dtype: object

In [30]:
import string
translator = str.maketrans('' , '' , string.punctuation)

In [31]:
quotes = quotes.apply(lambda x : x.translate(translator))

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [33]:
vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(quotes)

In [34]:
word_index = tokenizer.word_index
print(len(word_index))

248


In [35]:
sequence = tokenizer.texts_to_sequences(quotes)


In [36]:
X = []
y = []
for seq in sequence:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[1])



In [38]:
len(X)

7656

In [39]:
max_len = max(len(x) for x in X)
max_len

23